In [1]:
# #Install pyliftover if you haven’t yet:
# !pip install pyliftover

In [2]:
# Chromatin State Analysis
# !pip install pybedtools

In [1]:
import os
#os.environ["BEDTOOLS_PATH"] = "/share/lasallelab/Ensi/anaconda3/allcools/bin/bedtools"  # Adjust this path to where BEDTools is installed on your system.
import pybedtools
import pandas as pd
from pyliftover import LiftOver
# the same code to fix the error;
import requests
import gzip
import shutil


In [3]:
# downalod the liftover bedfile

# !wget https://hgdownload.soe.ucsc.edu/gbdb/rheMac10/liftOver/rheMac10ToHg38.over.chain.gz
# !wget https://encodeproject.org/files/ENCFF001XFG/@@download/ENCFF001XFG.bed.gz #chromatin state human

In [4]:
# !conda activate /share/lasallelab/Ensi/anaconda3/allcools/

In [ ]:

#lift over hyper nad hyper dmgs
# Define methylation contexts
contexts = ['hyper', 'hypo']

# Initialize the liftover converter
lo = LiftOver('../../../rheMac10ToHg38.over.chain.gz')

# Function to perform liftover and export to BED
def process_file(context):
    file_path = f'../homer/rheMac10/mcg/HOMER/DMRs_{context}.bed'  # Removed cluster-specific naming
    data = pd.read_csv(file_path, sep='\t', header=None)
    
    # Ensure correct column names
    data.columns = ["chrom", "start", "end"]
    human_coords = []

    for idx, row in data.iterrows():
        chrom, start, end = row['chrom'], row['start'], row['end']

        # Convert coordinates using liftover
        human_start = lo.convert_coordinate(chrom, start)
        human_end = lo.convert_coordinate(chrom, end)

        if human_start and human_end:
            start_new = min(human_start[0][1], human_end[0][1])
            end_new = max(human_start[0][1], human_end[0][1])
            human_coords.append([human_start[0][0], start_new, end_new])
        else:
            print(f"Region {chrom}:{start}-{end} could not be lifted over.")

    # Export results
    if human_coords:
        lifted_df = pd.DataFrame(human_coords, columns=['chrom', 'start', 'end'])
        output_bed_file = f'lifted_sigRegions_{context}_mcg.bed'
        lifted_df.to_csv(output_bed_file, sep='\t', index=False, header=False)
        print(f"File processed and saved as {output_bed_file}")
    else:
        print(f"No regions successfully lifted over for {context}.")

# Process each methylation context
for context in contexts:
    process_file(context)
